# Curva ROC ( Receiver Operating Characteristic )

## Modificación del umbral de decisión

- Un modelo probabilístico calcula la probabildad de que una observación pertenezca a una clase.
- Hasta ahora en clasificación binaria, modelamos $P(Y=1|X=X_i)$. Si esta probabilidad está por encima de 0.5, clasificamos a $X_i$ como perteneciente a la clase 1 o 0 en caso contrario.
- Una vez que realizamos la asignación de clases podemos armar la matriz de confusión, y de la matriz de confusión, calcular los diversos indicadores de performance del modelo.

¿Podemos armar distintas matrices de confusión, modificando el umbral de clasificación?

Podemos, y esto dará origen a distintos modelos, cada uno con su propia matriz de confusión y por lo tanto, distintos valores de sensibilidad, especificidad, etc. 

Armemos un dataset sintético para analizar cómo varían los parámetros de performance del modelo (calculados a partir de la matriz de confusión) en función del umbral.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import helper
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import SGDClassifier

In [ ]:
def generate_gaussians(N0,N1,mu0,mu1,s):
    # N0: Cantidad de muestras pertenecientes a la clase 0
    # N1: Cantidad de muestras pertenecientes a la clase 1
    # mu0: Media de las muestras pertenecientes a 0
    # mu1: Media de las muestras pertenecientes a 1
    # s: Desvío estandar de los predictores. Los predictores son independientes.
    sigma=[[s,0],[0,s]]
    np.random.seed(231)
    X=np.vstack([np.random.multivariate_normal(mu0, sigma,N0),np.random.multivariate_normal(mu1, sigma,N1)])
    y=np.hstack([np.zeros(N0),np.ones(N1)])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True)
    return X_train, X_test, y_train, y_test

## Intepretación del dataset

Supongamos que los valores de los predictores de este dataset son los valores de concentración en sangre de dos indicadores.  
En el siguiente gráfico los puntos azules representan valores de los indicadores en personas enfermas.  
Los puntos rojos representan valores de los indicadores en personas sanas.

In [ ]:
X_train, X_test, y_train, y_test=generate_gaussians(N0=1000,N1=1000,mu0=[2,5],mu1=[3,3],s=1)

In [ ]:
helper.plotBoundary(X_train,y_train)
helper.plotBoundary(X_test,y_test)

Escribir una función plot_metrics que reciba:

- y_true: Las anotaciones de las clases de cada observacion. El ground truth.
- y_proba: La probabilidad que dió el clasificador de que cada observación pertenezca a la clase 1
- num: cantidad de valores que tomará el threshold
- plot_metrics: si es true, se graficarán las métricas en función del threshold
- plot_ROC: si es true, se graficará la curva ROC

La función devuelve la binary cross entropy y el área bajo la curva ROC para los datos de entrada.

In [ ]:
# La siguiente función grafica las métricas de la matriz de confunsión y la curva ROC.
# Devuelve el area bajo la curva ROC y la loss
def plot_metrics(y_true,y_proba,num=50,plot_metrics=False,plot_ROC=False):
    P= #cantidad de observaciones pertenecientes a la clase P (1)
    N= #cantidad de observaciones pertenecientes a la clase N (0)
    if plot_metrics:
        cond_pos= #Proporción de observaciones positivas (PRIORI)
        cond_neg= #Proporción de observaciones negativas (PRIORI)
        # Reserva de espacio para las métricas
        accuracy=np.zeros(num)
        specificity=np.zeros(num)
        ppv=np.zeros(num)
        npv=np.zeros(num)
        fnr=np.zeros(num)
    sensitivity=np.zeros(num+2) # Se reservan dos lugares mas, para fijar lo thresholds en cero y uno
    fpr=np.zeros(num+2)  # Se reservan dos lugares mas, para fijar lo thresholds en cero y uno
    thresholds=np.linspace(0.01,0.99,num)
    for idx,threshold in enumerate(thresholds):
        y_pred=1*(y_proba>threshold)
        conf_matrix=metrics.confusion_matrix(y_true, y_pred, labels=[0,1])
        TN=conf_matrix[0,0]
        FN=conf_matrix[1,0]
        FP=conf_matrix[0,1]
        TP=conf_matrix[1,1]
        sensitivity[idx+1]= #Completar
        fpr[idx+1]= #Completar
        if(plot_metrics):
            accuracy[idx]= #Completar
            specificity[idx]= #Completar
            ppv[idx]= #Completar
            npv[idx]= #Completar
            fnr[idx]= #Completar
    # Completo los valores de los extremos.
    sensitivity[0]=1 
    sensitivity[-1]=0
    fpr[0]=1
    fpr[-1]=0
    roc_auc = metrics.auc(fpr, sensitivity)
    if(plot_metrics):
        # Gráfica de las métricas en función del threshold
        plt.figure(figsize=[15,10])
        plt.plot(np.linspace(0,1,num),accuracy, label='Accuracy')
        plt.plot(np.linspace(0,1,num),sensitivity[1:-1], label='Sensitivity or True Positive Rate or Recall')
        plt.plot(np.linspace(0,1,num),specificity, label='Specificity')
        plt.plot(np.linspace(0,1,num),ppv, label='Positive Predictive Value or Precision ')
        plt.plot(np.linspace(0,1,num),npv, label='Negative Predictive Value')
        plt.plot(np.linspace(0,1,num),fnr, label='False Negative Rate or Miss Rate')
        plt.plot(np.linspace(0,1,num),fpr[1:-1], label='False Positive Rate or Fall-out')
        plt.legend()
        plt.ylabel('Metric')
        plt.xlabel('Threshold')
        plt.show()
    if(plot_ROC):
        plt.plot(fpr,sensitivity,label="ROC curve. Area={}".format(roc_auc))
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.show()
    cant_obs=len(y_true)
    epsilon=1e-9 # este valor está para evitar el logaritmo de cero
    loss=  #Completar
    return roc_auc,loss

Ahora utilice el siguiente código para generar observaciones y graficar las métricas solicitadas para el caso: 

$$N1=10000,N2=2000,mu1=[-4,4],mu2=[4,-4],s=20$$

In [ ]:
X_train, X_test, y_train, y_test=generate_gaussians(N0=10000,N1=2000,mu0=[-4,4],mu1=[4,-4],s=20)
clf=SGDClassifier(loss='log',penalty=None,max_iter=1000,tol=1e-6,learning_rate='adaptive',eta0=0.1)
clf.fit(X_train,y_train)
y_train_pred_proba=clf.predict_proba(X_train)[:,1]
plot_metrics(y_train,y_train_pred_proba,plot_metrics=True,num=100)

### Preguntas:

- Cuánto vale la binary crossentropy?  
- Cuánto vale el área bajo la curva ROC?  
- Si elegimos un threshold de 0.2:  
    -  ¿Cuánta gente que se haga el test y le haya dado positivo,no tiene la enfermedad?
    -  ¿Cuánta gente que cuyo test de negativo, tiene en realidad la enfermedad?
    -  ¿Cuánta gente que cuyo test de positivo, tiene efectivamente la enfermedad?
    -  ¿Cuánta gente que cuyo test de negativo, no tiene la enfermedad?
    
- Si elegimos un threshold de 0.2:  
    -  ¿Cuánta gente que se haga el test y le haya dado positivo, no tiene la enfermedad?
    -  ¿Cuánta gente que cuyo test de negativo, tiene en realidad la enfermedad?
    -  ¿Cuánta gente que cuyo test de positivo, tiene efectivamente la enfermedad?
    -  ¿Cuánta gente que cuyo test de negativo, no tiene la enfermedad?

### Problema (se alienta la resolución en conjunto):

Usted es empleado de un sistema de salud que utiliza el test anterior en pacientes para detectar una enfermedad. 

- El costo de realizar este test (de descarte) es despreciable para el sistema de salud.  
- Si alguien se hace el test de descarte y da positivo, tiene que realizarse un test mas caro pero definitivo (detección perfecta) cuyo costo es 2500 pesos. 
- Si el test definitivo da positivo, el paciente deberá realizarse un tratamiento (de detección temprana) cuyo costo es de 2500 pesos.  
- Si el test definitivo da negativo, el paciente no deberá realizarse ningún tratamiento, ya que está sano.
- Si el test de descarte da negativo, pero el paciente estaba enfermo, mas adelante la enfermedad se detectará y deberá realizar un tratamiento (de detección tardía) cuyo costo es de 9500 pesos.
- Si el test de descarte da negativo, y el paciente no estaba enfermo, no hay ningún costo asociado para la empresa.

Pensando solamente en el costo total para el sistema de salud, es decir, cuánto dinero promedio se gasta por paciente, ¿cuál es valor de threshold que minimiza este costo? 

Recomendación: Exprese el costo en función de las distintas métricas estudiadas en ingréselo en la función anterior como una métrica mas. Luego utilice argmin sobre el arreglo de esta nueva métrica para encontrar el threshold que minimiza el costo.

¿Cuánto vale la sensibilidad, la especificidad, el VPP y el VPN del test de descarte para este threshold?

In [ ]:
plt.figure(figsize=[20,12])
auc_rocs_train=list()
losses_train=list()
auc_rocs_test=list()
losses_test=list()
plots=4
aux=5
res=plots*aux
grid = plt.GridSpec(plots-1, plots, wspace=0.4, hspace=0.3)
sigmas=np.geomspace(3,50,res)
plot_number=0
for idx,s in enumerate(sigmas):
    X_train, X_test, y_train, y_test=generate_gaussians(N1=1000,N2=20000,mu1=[-4,4],mu2=[4,-4],s=s)
    clf=SGDClassifier(loss='log',penalty=None,max_iter=1000,tol=1e-6,learning_rate='adaptive',eta0=0.1)
    clf.fit(X_train,y_train)
    y_train_pred_proba=clf.predict_proba(X_train)[:,1]
    roc,loss=plot_metrics(y_train,y_train_pred_proba,plot_ROC=False,num=10)
    auc_rocs_train.append(roc)
    losses_train.append(loss)
    y_test_pred_proba=clf.predict_proba(X_test)[:,1]
    roc,loss=plot_metrics(y_test,y_test_pred_proba,num=10)
    auc_rocs_test.append(roc)
    losses_test.append(loss)
    if idx % aux == 0:
        ax=plt.subplot(grid[(plots-2):plots-1, plot_number:(plot_number+1)])
        helper.plotBoundary(X_test,y_test,clf,ax=ax,mins=[-15,-15],maxs=[15,15])
        ax.set_xlim([-15,15])
        ax.set_ylim([-15,15])
        acc=clf.score(X_test,y_test)
        ax.set_title("sigma={:.2f}, test_accuracy={:.2f}".format(s,acc))
        plot_number += 1
plt.subplot(grid[:-1,:])
plt.plot(auc_rocs_train,losses_train,label="train",c="b")
plt.plot(auc_rocs_test,losses_test,label="test",c="r")
plt.scatter(auc_rocs_train,losses_train,c="b",label="train_sigma")
plt.scatter(auc_rocs_test,losses_test,c="r",label="test_sigma")
for idx,s in enumerate(sigmas):
    plt.text(auc_rocs_train[idx],losses_train[idx],"{:.2f}".format(s),color="b")
    plt.text(auc_rocs_test[idx],losses_test[idx],"{:.2f}".format(s),color="r")
plt.legend()
plt.xlabel("AUC_ROC")
plt.ylabel("LOSS")
plt.show()